<a href="https://colab.research.google.com/github/TerriDonut/Yandex-educational-projects/blob/main/big_data_processing_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

____    
    
**Проект: Системы обработки больших данных**
_____
**Описание и цель проекта**

В проекте нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году.
______

**Исходные данные.**

В колонках датасета содержатся следующие данные:

- longitude — широта;
- latitude — долгота;
- housing_median_age — медианный возраст жителей жилого массива;
- total_rooms — общее количество комнат в домах жилого массива;
- total_bedrooms — общее количество спален в домах жилого массива;
- population — количество человек, которые проживают в жилом массиве;
- households — количество домовладений в жилом массиве;
- median_income — медианный доход жителей жилого массива;
- median_house_value — медианная стоимость дома в жилом массиве;
- ocean_proximity — близость к океану.
____

**Задачи проекта.**

На основе данных нужно предсказать медианную стоимость дома в жилом массиве — median_house_value. Обучить модель и сделать предсказания на тестовой выборке. Для оценки качества модели использовать метрики RMSE, MAE и R2.
____

**Содержание проекта.**

***Инициализация локальной Spark-сессии:***

***Чтение содержимого файла:***

***Вывод типов данных колонок датасета. Используя методы pySpark***

***Предобработка данных***
* [Исследование данных на наличие пропусков и заполнение их](#1-bullet)
* [Преобразование колонок с категориальными значениями техникой One hot encoding](#2-bullet)

***Построение двух моделей линейной регрессии на разных наборах данных***
* [Используя все данные из файла](#3-bullet)
* [Используя только числовые переменные, исключив категориальные](#4-bullet)

***Сравнение результатов работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2***

***Общий вывод***


## Инициализация локальной Spark-сессии

In [ ]:
import sys
!"{sys.executable}" -m pip install pyspark -q

In [ ]:
import pandas as pd
import numpy as np

from pyspark.ml.regression import LinearRegression

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator

from warnings import simplefilter

In [ ]:
simplefilter(action='ignore', category=FutureWarning)


spark = SparkSession \
    .builder \
    .master('local')\
    .appName('Housing - LinearRegression') \
    .getOrCreate()

## Чтение содержимого файла

In [ ]:
try:
    df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True)
except:
    df = spark.read.option('header', 'true').csv('/content/housing.csv', inferSchema = True)

## Вывод типов данных колонок датасета. Используя методы pySpark

In [ ]:
df.printSchema()
df.show(5)

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR B

## Предобработка данных

### Исследование данных на наличие пропусков и заполнение их
<a id='1-bullet'></a>

In [ ]:
for column in df.columns:
    miss_val_count = df.where(F.col(column).isNull()).count()
    print(f'Кол-во пропущенных значений в {column}: {miss_val_count}')

Кол-во пропущенных значений в longitude: 0
Кол-во пропущенных значений в latitude: 0
Кол-во пропущенных значений в housing_median_age: 0
Кол-во пропущенных значений в total_rooms: 0
Кол-во пропущенных значений в total_bedrooms: 207
Кол-во пропущенных значений в population: 0
Кол-во пропущенных значений в households: 0
Кол-во пропущенных значений в median_income: 0
Кол-во пропущенных значений в median_house_value: 0
Кол-во пропущенных значений в ocean_proximity: 0


<b>Комментарий к пропускам</b>

Пропуски замечены только в количестве спальных комнат. Речь в данных идёт о жилых массивах, поэтому сослаться на пропуск в случае студии невозможно (мало вероятно что весь жилой массив состоит только из квартир данного типа)

Заполню пропуски медианой

In [ ]:
median_total_bedrooms = df.approxQuantile('total_bedrooms', [0.5], 0)[0]

df = df.fillna({'total_bedrooms': median_total_bedrooms})
print('Кол-во пропусков в total_bedrooms: ', df.where(df['total_bedrooms'].isNull()).count())


Кол-во пропусков в total_bedrooms:  0


### Преобразование колонок с категориальными значениями техникой One hot encoding
<a id='2-bullet'></a>

In [ ]:
train_data, test_data = df.randomSplit([0.7, 0.3], seed=2022)

pipeline = Pipeline(stages=[
    StringIndexer(inputCol='ocean_proximity', outputCol='ocean_index', handleInvalid='skip'),

    OneHotEncoder(inputCol='ocean_index', outputCol='ocean_oh'),

    VectorAssembler(inputCols=['ocean_oh'], outputCol='categorical_features'),

    VectorAssembler(inputCols=['longitude', 'latitude', 'housing_median_age',
                              'total_rooms', 'total_bedrooms', 'population',
                              'households', 'median_income'],
                    outputCol='numerical_features'),

    StandardScaler(inputCol='numerical_features', outputCol='numerical_features_scaled')
])

model = pipeline.fit(train_data)

train_transformed = model.transform(train_data)
test_transformed = model.transform(test_data)

train_transformed.show(2)



+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-----------+-------------+--------------------+--------------------+-------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_index|     ocean_oh|categorical_features|  numerical_features|numerical_features_scaled|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-----------+-------------+--------------------+--------------------+-------------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|        2.0|(3,[2],[1.0])|       [0.0,0.0,1.0]|[-124.35,40.54,52...|     [-62.132722572510...|
|  -124.27|   40.69|              36.0|     2349.0|         528.0|    11

## Построение двух моделей линейной регрессии на разных наборах данных

### Используя все данные из файла
<a id='3-bullet'></a>

In [ ]:
lr = LinearRegression(featuresCol='features', labelCol='median_house_value')

full_as = VectorAssembler(inputCols=['categorical_features','numerical_features_scaled'],
                          outputCol="features")

train_transformed_full = full_as.transform(train_transformed)
test_transformed_full = full_as.transform(test_transformed)

model_lr = lr.fit(train_transformed_full)

pred = model_lr.transform(test_transformed_full)

### Используя только числовые переменные, исключив категориальные
<a id='4-bullet'></a>

In [ ]:
num_as = VectorAssembler(inputCols=['numerical_features_scaled'], outputCol="features")

train_transformed_num = num_as.transform(train_transformed)
test_transformed_num = num_as.transform(test_transformed)

In [ ]:
model_num = lr.fit(train_transformed_num)

pred_num = model_num.transform(test_transformed_num)

## Сравнение результатов работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2

In [ ]:
ev_rmse = RegressionEvaluator(
    labelCol='median_house_value',
    predictionCol='prediction',
    metricName='rmse'
)

ev_mae = RegressionEvaluator(
    labelCol='median_house_value',
    predictionCol='prediction',
    metricName='mae'
)

ev_r2 = RegressionEvaluator(
    labelCol='median_house_value',
    predictionCol='prediction',
    metricName='r2'
)

rmse = ev_rmse.evaluate(pred)
mae = ev_mae.evaluate(pred)
r2 = ev_r2.evaluate(pred)

rmse_num = ev_rmse.evaluate(pred_num)
mae_num = ev_mae.evaluate(pred_num)
r2_num = ev_r2.evaluate(pred_num)

print(f'RMSE: Полных данных {rmse:.2f}, Числовых {rmse_num:.2f}')
print(f'MAE: Полных данных {mae:.2f}, Числовых {mae_num:.2f}')
print(f'R2: Полных данных {r2:.3f}, Числовых {r2_num:.3f}')

RMSE: Полных данных 68238.23, Числовых 69095.77
MAE: Полных данных 49890.88, Числовых 50881.56
R2: Полных данных 0.650, Числовых 0.641


In [ ]:
spark.stop()

## Общий вывод

<b>Выполняемая задача</b>

Обучение модели линейной регрессии на данных о жилье в Калифорнии в 1990 году и
предсказание медианной стоимости дома в жилом массиве.

<b>Итоги обучения и оценки моделей</b>

- Значение RMSE для числовых данных выше, чем для полных данных, модель работает хуже при работе с числовыми данными по сравнению с полными данными.

- Аналогично случаю с RMSE, значение MAE для числовых данных выше, чем для полных данных

- Значение R2 для полных данных немного выше, чем для числовых данных, что указывает на незначительную, но важность кат.данных

В целом, модель имеет большой разброс и низкое качество, что говорит о необходимости ввода новых признаков или урезке после корреляционного анализа самых малозначемых для таргета.